## Análise e pré processamento dos dados da base do Titanic

### A base traz informações dos passageiros que estavam no navio no dia do acidente:

https://www.kaggle.com/datasets/yasserh/titanic-dataset?select=Titanic-Dataset.csv

As colunas desse dataset são:
- Passenger ID: ID do passageiro (número único para cada um dos passageiros)
- Survived: sobrevivente (0 = Não, 1 = Sim)
- Pclass: Classe da passagem (1 = primeira classe, 2 = segunda classe, 3 = terceira classe)
- Name: nome do passageiro
- Sex: Gênero do passageiro
- Age: Idade (em anos) do passageiro
- SibSp: número de irmãos / cônjuges a bordo do Titanic
- Parch: número de pais / filhos a bordo do Titanic
- Ticket: número do ticket
- Fare: tarifa da passagem
- Cabin: número da cabine
- Embarked: porto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)

In [10]:
from pyspark.sql import SparkSession, functions as F

spark = (
    SparkSession
    .builder
    .appName('Titanic')
    .getOrCreate()
)
spark

In [2]:
df_titanic = (
    spark.read.format('csv')
    .option('inferSchema', True)
    .option('header', True)
    .load('data/Titanic-Dataset.csv')
)

df_titanic.printSchema()
df_titanic.show(5)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|   

In [3]:
# Total Linhas e Colunas

print((df_titanic.count(), len(df_titanic.columns)))

(891, 12)


In [4]:
df_titanic.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [5]:
# Localizando e contando todas as colunas do Dataframe com valores: 
# - Null, None, NaN 
# Caso haja necessidade de especificar colunas: 
#  - df_Columns=["col1","col2","col3"]

df_titanic.select(
    [
        F.count(F.when(F.isnan(c) 
        | F.col(c).isNull(), c)).alias(c) 
        for c in df_titanic.columns]
).show()


+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [6]:
# Localizando e contando valores: empty, None, Null, Nan com string literais de todas as Colunas do Dataframe

df_titanic.select(
    [F.count(F.when(F.col(c).contains('None') | 
                            F.col(c).contains('NULL') | 
                            (F.col(c) == '' ) | 
                            F.col(c).isNull() | 
                            F.isnan(c), c 
                           )).alias(c)
                    for c in df_titanic.columns]
).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [7]:
# Find high cardinality
df_titanic.groupBy('name').count().count()

891

#### Verificando Cardinalidade

In [8]:
list = []
for col in df_titanic.columns:
    data = (col, int( df_titanic.select(col).distinct().count())  )
    list.append(data)


(
    spark.createDataFrame(list, ["cols","total"])
    .orderBy(F.col('total').desc())
    .show()
)    

+-----------+-----+
|       cols|total|
+-----------+-----+
|PassengerId|  891|
|       Name|  891|
|     Ticket|  681|
|       Fare|  248|
|      Cabin|  148|
|        Age|   89|
|      SibSp|    7|
|      Parch|    7|
|   Embarked|    4|
|     Pclass|    3|
|   Survived|    2|
|        Sex|    2|
+-----------+-----+



In [9]:
# Verificando a quantidade de pessoas que sobreviveram separando pelo sexo

df_titanic.groupBy('Sex', 'Survived').count().orderBy(F.desc('Sex')).show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|  male|       1|  109|
|female|       1|  233|
|female|       0|   81|
+------+--------+-----+

